# Challenge 5 ou mini-projet de Sécurité

Boisel Jules
Xu William
Savadoux Baptiste

In [665]:
import csv
import requests
import datetime
import feedparser
import uuid

import os

In [666]:
if not os.path.exists("Csv_Site"): # créer les fichiers qu'on a besoin pour faire fonctionner le programme
    os.makedirs("Csv_Site")
if not os.path.exists("result"):
    os.mkdir("result")

In [667]:
#fonction qui permet de supprimer les constant \n \r \f \t ...

def enlever(text):
    return text.replace("\n", "").replace("\t", "").replace("\b", "").replace("\r", "").replace("\a", "").replace("\v", "").replace("  ","").replace(" ","").rstrip()

def enleverbis(text):
    return text.replace("\t", "").replace("\b", "").replace("\r", "").replace("\a", "").replace("\v", "").replace("  ","").replace(" ","").rstrip()


#input : "  J'aime les \t\t\t pates a la va  nille "

#output : "J'aime les pates a la vanille"



<img src="https://cdn.discordapp.com/attachments/1031485777353375761/1035492748918276146/download.png" style="width:400px;height:500px">

In [668]:
#cette fonction nous permet d'ecrire dans un csv les sites ayant le rss ou le atom

def New_CSV_site(emplacement , list_site):  # ecrit dans un csv (c'est juste plus simple)
    with open(emplacement, "w", newline='', encoding='utf-8') as f:
        for i in list_site:
            f.write(i)
    # print("terminé",emplacement)

#allez voir le fichier site_bon_rss et site_bon_atom

In [669]:
liste_site_bon_rss = [] # définition d'une variable global pour pouvoir ajouter les sites dans les 2 fonctions
liste_site_bon_atom = []


# vérifie s'il y a "atom" dans un site (même fonction qu'au-dessus mais pour atom)

def Verification_atom(req, site):
    global liste_site_bon_atom

    #on fait une requete pour avoir le html
    #on split par saut de ligne
    split = req.text.split("\n")

    #si la ligne contient atom on a joute a la variable tableau global liste_site_bon_atom
    for i in split:
        if "atom" in i.lower() or "atoms" in i.lower() :
            if str(site + "\n") not in liste_site_bon_atom:
                liste_site_bon_atom.append(site + "\n")

In [670]:
#cette fonction permet de recuperer les liens comportant rss selon la balise <a ... href="....">
def recuperation_lien_rss(lien_http):  # recupere le lien du rss

    #check si on peut faire une requete
    try:
        st = enlever(requests.get(lien_http).text.replace("'", '"'))
    except:
        #sinon on return pr arreter la fonction
        return

    #variables pour contenir les balises et lien récupéré
    balise_complet,lien = "",""
    #booleans pour savoir quand recuperer les letters etc
    bool_balise,balise_lien = False,False
    # temp = []
    for j in range(len(st)-1):  # on check char par char

        #Check si c'est le debut d'une balise <a ... >
        if st[j] + st[j + 1] == "<a":
            bool_balise = True

        #check si c'est une balise fermante de <a ... > soit </a> on met bool_balise a false pour arreter de recuperer les idnex
        if (st[j - 4] + st[j - 3] + st[j - 2] + st[j - 1] == "</a>") and bool_balise:  # check si fin
            bool_balise = False
            if "rss" in balise_complet.lower() and "#" not in balise_complet:

                #parfois c'est juste le /feed/ etc sans le lien devant

                if "http" not in lien:
                    if lien_http+lien[1:]+"\n" not in liste_site_bon_rss:
                        liste_site_bon_rss.append((lien_http + lien[1:] + "\n"))

                else:
                    if lien + "\n" not in liste_site_bon_rss:
                        liste_site_bon_rss.append((lien + "\n"))

                # temp += lien,
            balise_complet,lien = "",""

        ####### Dans la var lien #######
        #permet de pouvoir recuperer les index utiles Si True
        if st[j - 6] + st[j - 5] + st[j - 4] + st[j - 3] + st[j - 2] + st[j - 1] == 'href="':
            balise_lien = True
        #Si false, ne permet de plus de recuprer les index
        if st[j] == '"' and st[-1] != "=":  # check si fin
            balise_lien = False

        #Ecriture si balise == True
        if bool_balise:
            #dans la balise complet
            balise_complet += st[j]
            if balise_lien:
                #le lien
                lien += st[j]

In [671]:
# recuperation_lien_rss("https://www.cert.ssi.gouv.fr/")

In [672]:

#recupere les lignes atoms dans un csv
def recuperation_atom():
    q =  open("Csv_Site/line_atom.csv", "w", newline='', encoding='utf-8')
    with open("Csv_Site/site_bon_atom.csv", "r") as f: # boucle dans le fichier csv ou il y a les sites possedant des lien atom
        readCSV = csv.reader(f)
        for i in readCSV: # on fait tourner i dans le csv (ici i va être égale à chaque lettre du fichier)
            print ('lien en cours : ' , i)
            req = requests.get("".join(i)) # envoie une requête au site du fichier csv
            split = enleverbis(req.text).split("\n") # on split la chaine à chaque fois qu'il y a un retour à la ligne pour sépararer chaque ligne de l'html
            for j in split: # on fait tourner j dans le split (ici j va être égale a chaque ligne du fichier)
                if "atom" in j: # si il y a atom dans la ligne html
                    q.write(j + "\n") # si on trouve atom, on ajoute la ligne avec un retour à la ligne a la fin dans la liste
    q.close()


In [673]:
# recuperation_atom()

In [674]:
liste_lien_atom = []

def recuperation_lien_atom():
    global liste_lien_atom
    ecriture = open ("Csv_Site/lien_atom.csv", "w", newline='', encoding='utf-8')
    with open("Csv_Site/line_atom.csv", "r") as f:
        f = ''.join(f.readlines())
        # print(list(map(str,f)))
        balise_complet,balise_lien = "",""
        bool_complet,bool_lien = False,False
        # temp = []
        for i in range(len(f) - 4):


            # si on croise
            if f[i - 2] + f[i - 1] == "<a":
                bool_complet = True
                # si on croise  on arrete l'ecriture avec g==false ou si on croise "> er que g==True pour eviter les div

            if (f[i - 4] + f[i - 3] + f[i - 2] + f[i - 1] == "</a>" or (f[i - 2] + f[i - 1] == '">') and bool_complet == True):  # check si fin
                bool_complet = False
                if "atom" in balise_complet.lower() and "#" not in balise_complet:  # eviter les les #
                    # print(l.index("atom"))
                    tempo = balise_complet.index("atom")
                    # si    [pas de lettre ou de chiffre]atom[pas de lettre ou de chiffre] ou [pas de lettre ou de chiffre]atoms[pas de lettre ou de chiffre]
                    if (balise_complet[tempo - 1].isalnum() == False and balise_complet[tempo + 4].isalnum() == False) or (
                            balise_complet[tempo - 1].isalnum() == False and balise_complet[tempo + 5].isalnum() == False and balise_complet[
                        tempo + 4] == "s"):
                        # print(l)
                        # temp += balise_lien,
                        #ecriture.write(balise_lien+"\n")
                        if balise_lien+'\n' not in liste_lien_atom:
                            liste_lien_atom.append(balise_lien+'\n')
                        # if balise_lien != "":
                        #     liste_site_bon_atom.append(balise_lien)
                        # print ('lien en cours : ' , balise_lien)
                # print(l)
                # on renitialise tout
                balise_complet = ""
                balise_lien = ""
                # si on croise href on met h true afin de pouvoir ecrire
            if f[i - 6] + f[i - 5] + f[i - 4] + f[i - 3] + f[i - 2] + f[i - 1] == 'href="':
                bool_lien = True
                # si on croise " et != = on met h false afin d'arreter l'ecriture
            if f[i] == '"' and f[i + 1] != "=":  # check si fin
                bool_lien = False
            if bool_complet:  # g == True on ecrit
                balise_complet += f[i]
                if bool_lien:  # h == True on ecrit
                    balise_lien += f[i]

    ecriture.close()
    with open ("Csv_Site/lien_atom.csv", "w", newline='', encoding='utf-8') as f:
        for j in liste_lien_atom:
            f.write(j)


In [675]:
# recuperation_lien_atom()
# print(liste_site_bon_atom)
# recuperation_lien_rss("https://www.cert.ssi.gouv.fr/feed/")
# print(liste_site_bon_atom)
# print(liste_site_bon_rss)

In [676]:
#fonction qui retire les balises dans des balises <description>
#par ex : <description>J'aime les aptes a <p> vanille</p>!!!</description>
#la fonction enlevera la balaise <p> et </p>
def refra(court):
    resultat=""
    des_bool_sign = True
    court = enlever(court)
    for l in range(len(court) - 3):
        if court[l] == "<":
            des_bool_sign = False
        if court[l] == ">":
            des_bool_sign = True
        if des_bool_sign and (court[l].isalnum() or court[l] not in "<>{}[]"):
            if court[l] + court[l + 1] + court[l + 2] + court[l + 3] == "http":
                resultat += " "
            resultat += court[l]
    return resultat


In [677]:
#cette fonction nous verifier si on peut acceder au site, si oui on regarde si il y a rss ou atom avec la fonction 'recuperation_lien_rss' et 'verificatin_atom'
def Connexion_site(dossier_csv):
    with open(dossier_csv, "r") as f:  # on ouvre un csv en parametre afin d'en examiner plusieurs
        readCSV = csv.reader(f)
        #on boucle sur tous les liens du fichier
        for i in readCSV:
            print("lien en cours", i)  # c'est juste pour verifier que le programme de beug pas

            #si requete impossible, on passe au suivant
            try:
                #on fait la requete
                req = requests.get("".join(i))  # "".join(i) permet de relier le lien du site, car sinon il est séparé et on ne peut pas le traiter # on récupère la requests du site auquel on se connecte

                #si 'response 200' on check pour rss et atom
                if req.status_code == requests.codes.ok:  # si le site répond par une réponse 200
                    # Verification_RSS(req, "".join(i))  # lance les fonctions au-dessus avec en paramètre la requests au site et le lien du site pour l'ajouter dans une liste afin de l'ajouter dans un csv par la suite
                    recuperation_lien_rss("".join(i))
                    Verification_atom(req, "".join(i))
                else:
                    print("ca marche pas : ", i)  # pour voir quel site ne fonctionne pas
            except:
                print("lien dead")


In [678]:

def recuperation_des_sites():

    #cette fonction nous verifier si on peut acceder au site, si oui on regarde si il y a rss ou atom avec la fonction 'recuperation_lien_rss' et 'verificatin_atom'
    Connexion_site(input("inserez le path de votre fichier"))

    #On separe bien les liens rss et atom dans les fichiers respectiv
    New_CSV_site("Csv_Site/site_bon_rss.csv", liste_site_bon_rss)
    New_CSV_site("Csv_Site/site_bon_atom.csv", liste_site_bon_atom)

    recuperation_atom()
    #on check si c'est bien 'atom' et non 'atomique' par ex
    #la fonction recuperation_atom recupere les liens possedant 'atom' ou 'atoms'
    recuperation_lien_atom()




    print("Terminé")

In [679]:
choix_recommencer = input("Voulez vous recommencer la recherche des sites avec du rss et atom ou utiliser les précédent ?('y' ou 'n')")
if choix_recommencer == "y":
    if not os.path.exists("Csv_Site/site_bon_atom.csv"):
        a = open("Csv_Site/site_bon_atom.csv", "w") # initialisation des 2 fichiers utiles
        a.close()
    if not os.path.exists("Csv_Site/site_bon_rss.csv"):
        b = open("Csv_Site/site_bon_rss.csv", "w")
        b.close()
    recuperation_des_sites()

lien en cours ['https://www.cert.ssi.gouv.fr/']
lien en cours ['https://www.ssi.gouv.fr/']
lien en cours ['https://www.zataz.com/']
lien en cours ['https://www.usine-digitale.fr/cybersecurite/']
lien en cours ['https://www.silicon.fr/tag/cybersecurite']
lien en cours ['https://www.itsecurityguru.org/']
lien en cours ['https://securityweekly.com/']
lien en cours ['https://www.infosecurity-magazine.com/']
lien en cours ['https://www.futura-sciences.com/tech/cybersecurite/actualites/']
lien en cours ['https://siecledigital.fr/cybersecurite/']
lien en cours ['https://www.lemondeinformatique.fr/securite-informatique-3.html']
lien en cours ['https://www.rtl.fr/sujet/cybersecurite']
lien en cours ['https://www.zdnet.fr/actualites/cybersecurite-3900046206q.htm']
lien en cours ['https://www.cyberocc.com/sinformer/actualites-cyber/']
lien en cours ['https://cyware.com/cyber-security-news-articles']
lien en cours ['https://thehackernews.com/']
lien en cours ['https://www.securityweek.com/']
lien 

In [680]:

#dictionnaire pour checker les mois en digits
#si on a 'Nov' on renvoit '11'
month = {'01': 'janauary',
            '02': 'february',
            '03': 'march',
            '04': 'april',
            '05': 'may',
            '06': 'june',
            '07': 'july',
            '08': 'august',
            '09': 'september',
            '10': 'october',
            '11': 'november',
            '12': 'december'}


#date_debut = datetime.datetime(2000, 10, 1)


#la fonction datee permet de généraliser tous les formats afin de l'utiliser avec feedparser
def datee(v, date_debut):

    #on ajoute un espace devant la date afin de pouvoir realiser cette condition:
    #if ((date[i-1].isdigit() or date[i-1].isalpha()==False) and date[i].isalpha() and (date[i+1].isdigit() or date[i+1].isalpha()==False)) == True
    #soit (date[i-1].isdigit() or date[i-1].isalpha()==False)
    #          ^                        ^
    #          |                        |
    date = " "+v.lower().replace("-"," ").replace("/"," ").replace(",","") #2022-10-14T14:30:45Z" en 2022 10 14T14:30:45Z
    #
    b= ""


    #remplace les lettres par des espaces "2022 10 14T14:30:45Z" en "2022 10 14 14:30:45"
    #
    for i in range(len(date)-1):
        if ((date[i-1].isdigit() or date[i-1].isalpha()==False) and date[i].isalpha() and (date[i+1].isdigit() or date[i+1].isalpha()==False)) == True:
            b+=" "
        else:
            b+=date[i]

    # print(b)

    #on re attribut les modifications dans la variable date
    date=b
    # print(date)

    #dictionnaire pour faciliter l'utilisation et la lecture de la date
    current = {"annee": "", "mois": "", "jour": ""}

    f=[]
    c= False

    for i in range(len(date)-4):


        ##################### Step 1 ########################
        #check si c'est le jour/mois soit possede 2 chiffres soit 01 --> janvier
        if date[i-1].isalnum()==date[i+2].isalnum()==False and date[i].isdigit()==date[i+1].isdigit()==True and (date[i+2] not in ":" and date[i-1] not in ":"):
            f+=date[i]+date[i+1],

        #check si c'est le jour/mois soit possede 1 chiffre     soit 01 --> janvier à 1 --> janvier
        if date[i-1].isalnum()==date[i+1].isalnum()==False and date[i].isdigit()==True and (date[i+1] not in ":" and date[i-1] not in ":"):
            f+=date[i]+date[i+1],

        ##################### Step 2 ########################
        #check si c'est l'année soit possede 4 chiffres
        if date[i-1].isalnum()==False and date[i].isdigit() and date[i+1].isdigit() and date[i+2].isdigit() and date[i+3].isdigit() and date[i+4].isalnum()==False and date[i+4] not in ":/-" and date[i-1] not in ":/-":
            f+=date[i]+date[i+1]+date[i+2]+date[i+3],


        #si on a des mois en lettre on converti et on insert direct dans le dictionnaire
        #donc Step 1 sera le jour et non le mois

        for j,k in month.items():
            if date[i]+date[i+1]+date[i+2] in k:
                # print(j)
                current["mois"] = j
                c=True
                #c == True afin de pas inserer la variable du Step 1 dans current['mois']


    #si len(f) < 0, cela veut dire qu'il y a pas eu de "date de publciation"
    if len(f) >0:
        #si il y a eu le mois en lettre
        if c:
            current["jour"]=f[0]
            current["annee"]=f[-1]

        #si il n'y a pas eu de mois en lettre
        else:
            # print(f)
        #Seulement si on respect le format aaaa/mm/jj
            current["annee"] = f[0]
            # current["mois"] = f[1]
            current["mois"] = f[1]
            current["jour"] = f[-1]
        #on convertit en format date

        #on converti les dates obtenus en format "datetime"
        date_voulu = datetime.datetime(int(current["annee"]), int(current["mois"]), int(current["jour"]))

        #on check si c'est > a date voulu
        if date_debut<date_voulu:
            return True
    else:
        return False

In [681]:
#CA TRADUIT LES TEXTES != Francais en Francais

#
def traducteur(phrase):

    #code src : depuis azure microsoft   https://github.com/MicrosoftTranslator
    # Add your key and endpoint
    #api key william azure translator creer de ma propre poche
    #api william : fb8ce793c54347e19cdd06eced15d4a6
    #api baptiste : c5dfb5eefde5419c8c9a6f2f6813add9
    key = "c4c193f46d964e90a9399f4f208a8d7f"
    endpoint = "https://api.cognitive.microsofttranslator.com"

    # location, also known as region.
    # required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
    location = "francecentral"

    path = '/translate'
    constructed_url = endpoint + path

    params = {
        'api-version': '3.0',
        'to': ['fr']
    }
    headers = {
        'Ocp-Apim-Subscription-Key': key,
        # location required if you're using a multi-service or regional (not global) resource.
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    # You can pass more than one object in body.
    body = [{
        'text': phrase
    }]
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    return response[0]['translations'][0]['text']


In [682]:
 # code avec librairie feedparser

# permet de récuperer les informations comme le titre, la description, la date et le lien de l'article qui comporte le mot recherché.
def parser_site(emplacement, mot, ecriture, date_voulue):
    compteur = 0

    writer = csv.writer(ecriture) # pour éviter de réécrire plusieur foi la même chose
    writer.writerow([f"mot saisi : {mot}"]) # on ecrit le mot dans la première ligne
    writer.writerow(["title", "description", "date", "link"]) # on écrit l'entête

    #on ouvre les fichiers contenant les liens rss et atom
    with open(emplacement, "r", encoding='utf-8') as f:
        #variable pour checker les doublons
        check_doublon=[]
        compteur_article = 0 # compteur pour savoir le nombre d'articles

        #lecteur des liens ligne par ligne
        reader = csv.reader(f)
        for i in reader:
            print("lien en cours :", i)
            lib_feedparser = feedparser.parse("".join(i)) # permet de parser nos sites
            # print(lib_feedparser)
            for j in range(len(lib_feedparser.entries)):

                #try and except, car lorsqu'il y a pas de description, cela arrete de le program
                #try avec description
                try:
                    # print(lib_feedparser.entries[j].description)
                    if lib_feedparser.entries[j].updated is not None and lib_feedparser.entries[j].description is not None and lib_feedparser.entries[j].title is not None and lib_feedparser.entries[j].link is not None :

                        #traduit le parse titre et description en francais et regarde si notre mot s'y trouve.
                        if mot.lower() in (rapid:=traducteur(refra(str(lib_feedparser.entries[j].title.lower())))) or mot.lower() in (rapid2:=traducteur(refra(str(lib_feedparser.entries[j].description.lower()))).replace(';', ",")): # on les mets en lower pour être sûr de la comparaison
                            data = rapid, rapid2, str(refra(lib_feedparser.entries[j].updated)) , refra(lib_feedparser.entries[j].link)
                            compteur_article += 1

                            #on regarde si elle ne se trouve pas dans check_doublon
                            if datee(lib_feedparser.entries[j].updated, date_voulue) and data not in check_doublon:
                                print(data)
                                # on ajoute dans data le titre traduit, la derscription traduit, la date d'update, et le lien de l'article
                                writer.writerow(data) # et ensuite on les ajoute dans le csv
                                check_doublon.append(data)
                                compteur+=1


                #sinon sans description
                except:
                    if lib_feedparser.entries[j].updated is not None and lib_feedparser.entries[j].title is not None and lib_feedparser.entries[j].link is not None :
                        if mot.lower() in (rapid3:=traducteur(refra(str(lib_feedparser.entries[j].title.lower())))): # on les mets en lower pour être sûr de la comparaison
                            data = rapid3,"pas de description", str(refra(lib_feedparser.entries[j].updated)) , refra(lib_feedparser.entries[j].link)
                            # print(lib_feedparser.entries[j].updated)
                            if datee(lib_feedparser.entries[j].updated, date_voulue) and data not in check_doublon:
                                # on ajoute dans data le titre traduit, la derscription traduit, la date d'update, et le lien de l'article
                                writer.writerow(data) # et ensuite on les ajoute dans le csv
                                check_doublon.append(data)
                                compteur+=1
        writer.writerow([f"Il y a {compteur_article} fois le mot recherché"])
        writer.writerow([f"Il y a {compteur} articles"])

        print(f"Il y a {compteur_article} fois le mot recherché")
        print(f"Il y a {compteur} articles")

In [683]:
# recuperation_des_sites()

In [684]:
#fait la recherche des rss atom dans tous jnos liens...

boolean = True
#ca tourne pour faire des recherches à la suite

choix_mot = input("Manuel ou avec le fichier prédéfinie ?( 'm' pour manuel et 'f' pour la liste) : ")

# pour checher avec le fichier de mots clef
if choix_mot == 'f':
    if os.path.exists("Csv_Site/fichiers_mots_clef.csv") and open("Csv_Site/fichiers_mots_clef.csv") is not None:
        annee_voulue = int(input("Entrez l'année jusqu'à quand les articles doivent au maximum datée "))
        mois_voulue = int(input("Entrez le mois jusqu'à quand les articles doivent au maximum datée "))
        jours_voulue = int(input("Entrez le jour jusqu'à quand les articles doivent au maximum datée "))
        date_voulue = datetime.datetime(annee_voulue, mois_voulue, jours_voulue)
        for i in open("Csv_Site/fichiers_mots_clef.csv", "r", newline='', encoding='utf-8'):
            # mot = str(input("Entrez le mot/thèmes que vous souhaitez")).lower() # pour faire une boucle tant qu'on sort pas pour afficher plusieurs mots
            mot = i.replace("\n", "")
            print(f"mot saisi : {mot}")


            ecriture_rss = open("result/"+mot+"_rss.csv", "w", newline='', encoding='utf-8') # on affilie a ecriture l'ouverture du fichier csv où on veut ecrire les resultats
            ecriture_atom = open("result/"+mot+"_atom.csv", "w", newline='', encoding='utf-8') # on affilie a ecriture l'ouverture du fichiercsv où on veut ecrire les resultats
            parser_site("Csv_Site/site_bon_rss.csv", mot, ecriture_rss, date_voulue) # on l'appelle 2 fois pour différencier le rss de l'atom
            parser_site("Csv_Site/lien_atom.csv", mot, ecriture_atom, date_voulue)


            ecriture_rss.close()
            ecriture_atom.close()
    else:
        print("pas de fichier de mots clef")

# pour chercher le mot qu'on veut
elif choix_mot == 'm':
    good = True
    fichier_mots = open("Csv_Site/fichiers_mots_clef.csv", "a", newline='', encoding='utf-8')
    verification_mots = open("Csv_Site/fichiers_mots_clef.csv", "r", newline='', encoding='utf-8')

    while boolean:
        mot = str(input("Entrez le mot/thèmes que vous souhaitez('q' pour quitter)")).lower() # pour faire une boucle tant qu'on sort pas pour afficher plusieurs mots

        if mot == "q" or mot == "":
            break
        else:
            print(f"mot saisi : {mot}")
            for i in verification_mots:
                if mot in i:
                    good = False
            if good:
                print("le mot ajouté : "+ mot)
                fichier_mots.write(mot+"\n")
            ecriture_rss = open("result/"+mot+"_rss.csv", "w", newline='', encoding='utf-8') # on affilie a ecriture l'ouverture du fichier csv où on veut ecrire les resultats
            ecriture_atom = open("result/"+mot+"_atom.csv", "w", newline='', encoding='utf-8') # on affilie a ecriture l'ouverture du fichier csv où on veut ecrire les resultats
            annee_voulue = int(input("Entrez l'année jusqu'à quand les articles doivent au maximum datée "))
            mois_voulue = int(input("Entrez le mois jusqu'à quand les articles doivent au maximum datée "))
            jours_voulue = int(input("Entrez le jour jusqu'à quand les articles doivent au maximum datée "))
            date_voulue = datetime.datetime(annee_voulue, mois_voulue, jours_voulue)
            parser_site("Csv_Site/site_bon_rss.csv", mot, ecriture_rss, date_voulue) # on l'appelle 2 fois pour différencier le rss de l'atom
            parser_site("Csv_Site/lien_atom.csv", mot, ecriture_atom, date_voulue)


        ecriture_rss.close()
        ecriture_atom.close()
    fichier_mots.close()
    verification_mots.close()

mot saisi : vulnérabilité
le mot ajouté : vulnérabilité
lien en cours : ['https://www.cert.ssi.gouv.fr/feed/']
('certfr-2022-avi-996 : vulnérabilité dans python 3 (07 novembre 20', 'ce bulletin d&rsquo,actualit&eacute, du cert-fr revient sur les vuln&eacute,rabilit&eacute,s significatives de la semaine pass&eacute,e pour souligner leurs criticit&eacute,s. il ne remplace pa', 'Mon, 07 Nov 2022 09:34:37 +0', ' https://www.cert.ssi.gouv.fr/avis/CERTFR-2022-AVI-9')
('certfr-2022-avi-995 : multiples vulnérabilités dans le noyau linux d’ubuntu (04 novembre 20', 'ce bulletin d&rsquo,actualit&eacute, du cert-fr revient sur les vuln&eacute,rabilit&eacute,s significatives de la semaine pass&eacute,e pour souligner leurs criticit&eacute,s. il ne remplace pa', 'Fri, 04 Nov 2022 14:20:14 +0', ' https://www.cert.ssi.gouv.fr/avis/CERTFR-2022-AVI-9')
('certfr-2022-avi-994 : multiples vulnérabilités dans vmware spring (04 novembre 20', 'ce bulletin d&rsquo,actualit&eacute, du cert-fr revient sur les vu